In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
import function.other_tools as otool
import function.file as file

In [2]:
folder_path = 'search/'
df_search_result = file.read_all_csv_files_in_folder('search/')

# 刪除重複景點
df_search_result = df_search_result.drop_duplicates(subset='name', keep='first')
df_search_result.reset_index(drop=True, inplace=True)
df_search_result.head(2)

,Unnamed: 0,keyword,name,current_time,rating,rating_count,category,address,introduce,Website,img,latitude,longitude
0,0,南投縣中寮鄉 景點推薦,中寮劉家梅園 休園 （一期一會明年再見）,2023-08-02 18:35:21,4.3,125.0,旅遊景點,頂坑巷12號,NaN,https://www.google.com/maps/place/%E4%B8%AD%E5...,https://lh5.googleusercontent.com/p/AF1QipPowN...,23.955327,120.823706
1,1,南投縣中寮鄉 景點推薦,剝皮寮歷史街區,2023-08-02 18:35:21,4.2,12238.0,歷史建築,康定路173巷,18 世紀房屋林立的城市街區,https://www.google.com/maps/place/%E5%89%9D%E7...,https://lh5.googleusercontent.com/p/AF1QipP-Th...,25.036838,121.502165


In [3]:
df_search_result.loc[:, 'stay_time'] = None
df_search_result = df_search_result.drop(['Unnamed: 0'],axis = 1)
df_search_result.head(2)

,keyword,name,current_time,rating,rating_count,category,address,introduce,Website,img,latitude,longitude,stay_time
0,南投縣中寮鄉 景點推薦,中寮劉家梅園 休園 （一期一會明年再見）,2023-08-02 18:35:21,4.3,125.0,旅遊景點,頂坑巷12號,NaN,https://www.google.com/maps/place/%E4%B8%AD%E5...,https://lh5.googleusercontent.com/p/AF1QipPowN...,23.955327,120.823706,None
1,南投縣中寮鄉 景點推薦,剝皮寮歷史街區,2023-08-02 18:35:21,4.2,12238.0,歷史建築,康定路173巷,18 世紀房屋林立的城市街區,https://www.google.com/maps/place/%E5%89%9D%E7...,https://lh5.googleusercontent.com/p/AF1QipP-Th...,25.036838,121.502165,None


In [8]:
 # 記錄開始時間
start_time = time.time() 

#---- driver 設定 ----
# s = Service("./geckodriver.exe")
s = Service("./chromedriver.exe")
driver = webdriver.Chrome(service=s)
count = 0
# 最大視窗
driver.maximize_window() 
for index,run_name in enumerate(df_search_result['name']):
    url ='https://www.google.com.tw/?hl=zh_TW'
    driver.get(url)
    # otool.timesleep_rand()
    otool.timesleep_rand()
    # 輸入查詢關鍵字
    try:
        search_box = driver.find_element(By.NAME, 'q')
        search_box.send_keys(run_name)
        otool.timesleep_rand()
    except:
        continue
    # 送出關鍵字
    max_try = 0
    while max_try<3:
        try:
            btn_search = driver.find_element(By.CLASS_NAME, "gNO89b")
            btn_search.click()
            break
        except:
            max_try +=1
            otool.timesleep_rand()
            print(df_search_result['name'][index])
            continue
    if  max_try == 3:
        continue
    otool.timesleep_rand()
    total_result = BeautifulSoup(driver.page_source, "html.parser")
    staytime = ""
    try:
        target_result = total_result.find('div',class_='TQc1id')
        staytime = target_result.find('span',class_='ffc9Ud').text
        print(staytime)
    except:
        pass
    else:
        df_search_result.loc[index, 'stay_time'] = staytime

    end_time = time.time()  # 記錄結束時間

    # 花費時間
    print(df_search_result['name'][index],end=" ")
    print('staytime: ',staytime,end=" ")
    count += 1
    print("(",count,") runtimes: ",otool.get_run_times(end_time,start_time))
    if count%100==0:
        df_search_result.to_csv('result/search_overview_addstaytime.csv',encoding='utf-8-sig')


df_search_result.to_csv('result/search_overview_addstaytime.csv',encoding='utf-8-sig')
#---- 關閉瀏覽器 ----
driver.close()


中寮劉家梅園 休園 （一期一會明年再見）  staytime:   ( 1 ) runtimes:  0 hr 0 min 9.55 sec 
剝皮寮歷史街區  staytime:   ( 2 ) runtimes:  0 hr 0 min 17.63 sec 
仙洞坪棋盤石  staytime:   ( 3 ) runtimes:  0 hr 0 min 26.29 sec 
中寮石龍宮  staytime:   ( 4 ) runtimes:  0 hr 0 min 34.47 sec 
南投龍鳳瀑布  staytime:   ( 5 ) runtimes:  0 hr 0 min 44.54 sec 
仙峰日月洞  staytime:   ( 6 ) runtimes:  0 hr 0 min 51.66 sec 
中寮鄉金龍山法華寺  staytime:   ( 7 ) runtimes:  0 hr 0 min 59.41 sec 
廣興紙寮  staytime:   ( 8 ) runtimes:  0 hr 1 min 7.17 sec 
紙教堂 Paper Dome (綠色生態旅遊、親子體驗教學、友善環境動物農場、必玩自然景點)推薦人氣 
紙教堂 Paper Dome (綠色生態旅遊、親子體驗教學、友善環境動物農場、必玩自然景點)推薦人氣 
紙教堂 Paper Dome (綠色生態旅遊、親子體驗教學、友善環境動物農場、必玩自然景點)推薦人氣 
彩蝶瀑布  staytime:   ( 9 ) runtimes:  0 hr 1 min 28.39 sec 
1.5 小時
六七一茶花園  staytime:  1.5 小時 ( 10 ) runtimes:  0 hr 1 min 36.63 sec 
1 小時
造紙龍手創館  staytime:  1 小時 ( 11 ) runtimes:  0 hr 1 min 43.61 sec 
45 分鐘
文武廟年梯步道  staytime:  45 分鐘 ( 12 ) runtimes:  0 hr 1 min 51.74 sec 
45 分鐘
南投望高寮觀景平台  staytime:  45 分鐘 ( 13 ) runtimes:  0 hr 1 min 59.75 sec 
八通關社寮古道  stayti

In [10]:
df=df_search_result[['name','stay_time']]
df.to_csv('result/search_addstaytime.csv',encoding='utf-8-sig')
df.head(20)

,name,stay_time
0,中寮劉家梅園 休園 （一期一會明年再見）,None
1,剝皮寮歷史街區,None
2,仙洞坪棋盤石,None
3,中寮石龍宮,None
4,南投龍鳳瀑布,None
5,仙峰日月洞,None
6,中寮鄉金龍山法華寺,None
7,廣興紙寮,None
8,紙教堂 Paper Dome (綠色生態旅遊、親子體驗教學、友善環境動物農場、必玩自然景點)...,None
9,彩蝶瀑布,None
